In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm
import torch.optim as optim
from thop import profile
import torchvision.datasets
import random
import matplotlib.pyplot as plt

In [2]:

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to tensors
])

# Load dataset
train_size = datasets.ImageFolder(root=r"C:\Users\aliof\OneDrive\Desktop\faculity project\processed resized data\augmented ali data\balanced_dataset\train", transform=transform)
val_size = datasets.ImageFolder(root=r"C:\Users\aliof\OneDrive\Desktop\faculity project\processed resized data\augmented ali data\balanced_dataset\valid", transform=transform)

# Create data loaders
train_loader = DataLoader(train_size, batch_size=42, shuffle=True)
val_loader = DataLoader(val_size, batch_size=42, shuffle=False)

# Load pre-trained EfficientNet model
base_model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=len(train_size.classes))

# Optionally, move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model.to(device)

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(base_model.parameters(), lr=0.00001)
num_epochs = 50

best_val_loss = float('inf')
patience = 5
counter = 0

# Training loop
for epoch in range(num_epochs):
    base_model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True) as t:
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = base_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            t.set_postfix(loss=running_loss / total, accuracy=100 * correct / total)
            
    # Validation loop
    base_model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = base_model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    # Calculate average validation loss and accuracy
    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * val_correct / val_total

    print(f'Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')

    # Early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        counter = 0
        # Save the best model
        torch.save(base_model.state_dict(), 'best_model_paper.pth')
    else:
        counter += 1

    if counter >= patience:
        print(f'Validation loss did not improve for {patience} consecutive epochs. Early stopping...')
        break

print('Training completed!')

Loaded pretrained weights for efficientnet-b0


Epoch 1/50: 100%|████████████████████████████████████████| 1143/1143 [09:55<00:00,  1.92it/s, accuracy=78, loss=0.0276]


Validation Loss: 0.1891, Accuracy: 96.63%


Epoch 2/50: 100%|█████████████████████████████████████| 1143/1143 [09:52<00:00,  1.93it/s, accuracy=95.1, loss=0.00515]


Validation Loss: 0.0633, Accuracy: 98.56%


...

Validation Loss: 0.0211, Accuracy: 99.61%


Epoch 19/50: 100%|███████████████████████████████████| 1143/1143 [09:52<00:00,  1.93it/s, accuracy=99.9, loss=0.000143]


Validation Loss: 0.0204, Accuracy: 99.64%


Epoch 20/50: 100%|███████████████████████████████████| 1143/1143 [09:53<00:00,  1.93it/s, accuracy=99.8, loss=0.000139]


Validation Loss: 0.0199, Accuracy: 99.69%
Validation loss did not improve for 5 consecutive epochs. Early stopping...
Training completed!


In [4]:
train_size = datasets.ImageFolder(root=r"C:\Users\aliof\OneDrive\Desktop\faculity project\processed resized data\augmented ali data\balanced_dataset\train", transform=transform)
# Get class labels
class_labels = train_size.classes

# Print class labels with their index
for i, label in enumerate(class_labels):
    print(f"Index {i}: Class Label - {label}")

Index 0: Class Label - Common wheat
Index 1: Class Label - Convolvulus arvensis
Index 2: Class Label - Cotton
Index 3: Class Label - Euphorbia peplus
Index 4: Class Label - Grass
Index 5: Class Label - Lolium multiflorum
Index 6: Class Label - Maize
Index 7: Class Label - Nutgrass
Index 8: Class Label - Purslane
Index 9: Class Label - Sesame
Index 10: Class Label - Sugar beet
Index 11: Class Label - Tomato


In [6]:
# Load pre-trained EfficientNet model
base_model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=12)

# Load trained model's state dictionary
state_dict = torch.load('best_model_paper.pth')

# Load state dictionary into the model's parameters
base_model.load_state_dict(state_dict)

# Move the model to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model.to(device)

# Define input tensor shape (batch_size, channels, height, width)
input_shape = torch.randn(1, 3, 224, 224).to(device)  # Assuming input images are RGB with size 224x224

# Count the number of parameters
total_params = sum(p.numel() for p in base_model.parameters())
trainable_params = sum(p.numel() for p in base_model.parameters() if p.requires_grad)

# Calculate FLOPS
flops, _ = profile(base_model, inputs=(input_shape,), verbose=False)

print("For EfficientNet:")
print(f"FLOPS: {flops / (10**6):.2f} million")
print(f"Number of parameters: {total_params / (10**6):.2f} million")
print(f"Number of trainable parameters: {trainable_params / (10**6):.2f} million")


Loaded pretrained weights for efficientnet-b0
For EfficientNet:
FLOPS: 27.03 million
Number of parameters: 4.02 million
Number of trainable parameters: 4.02 million


In [7]:


# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # Convert images to tensors
])

class_labels = train_size.classes

# Load dataset
val_size = datasets.ImageFolder(root=r"C:\Users\aliof\OneDrive\Desktop\faculity project\test\New folder", transform=transform)

# Create data loader for validation set
val_loader = DataLoader(val_size, batch_size=32, shuffle=False)

# Load pre-trained EfficientNet model
base_model = EfficientNet.from_pretrained('efficientnet-b0')

# Replace the classifier of the EfficientNet model with a new one matching the number of classes
num_classes = len(class_labels)
num_ftrs = base_model._fc.in_features
base_model._fc = nn.Linear(num_ftrs, num_classes)

# Load trained model
base_model.load_state_dict(torch.load('best_model_paper.pth'))

# Optionally, move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model.to(device)

# Define loss function
criterion = torch.nn.CrossEntropyLoss()

# Evaluate the model on the validation dataset
base_model.eval()
class_correct = [0] * num_classes
class_total = [0] * num_classes

with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = base_model(inputs)
        _, predicted = torch.max(outputs, 1)
        for i in range(len(labels)):
            label = labels[i]
            pred = predicted[i]
            if label == pred:
                class_correct[label] += 1
            class_total[label] += 1

# Calculate accuracy for each class
class_accuracy = [class_correct[i] / class_total[i] if class_total[i] != 0 else 0 for i in range(num_classes)]
for i in range(num_classes):
    print(f"Accuracy for class {class_labels[i]}: {class_accuracy[i] * 100:.2f}%")


Loaded pretrained weights for efficientnet-b0


Evaluating: 100%|████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  5.91it/s]

Accuracy for class Common wheat: 90.91%
Accuracy for class Convolvulus arvensis: 100.00%
Accuracy for class Cotton: 75.00%
Accuracy for class Euphorbia peplus: 21.43%
Accuracy for class Grass: 100.00%
Accuracy for class Lolium multiflorum: 100.00%
Accuracy for class Maize: 75.00%
Accuracy for class Nutgrass: 96.43%
Accuracy for class Purslane: 100.00%
Accuracy for class Sesame: 100.00%
Accuracy for class Sugar beet: 75.00%
Accuracy for class Tomato: 42.86%


In [8]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Define lists to store metrics
class_precision = []
class_recall = []
class_f1_score = []

# Evaluate the model on the validation dataset
base_model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = base_model(inputs)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculate precision, recall, and F1-score for each class
for i in range(num_classes):
    precision = precision_score(true_labels, predictions, labels=[i], average='micro')
    recall = recall_score(true_labels, predictions, labels=[i], average='micro')
    f1 = f1_score(true_labels, predictions, labels=[i], average='micro')
    class_precision.append(precision)
    class_recall.append(recall)
    class_f1_score.append(f1)
    print(f"Metrics for class {class_labels[i]}:")
    print(f"  Precision: {precision:.2f}")
    print(f"  Recall: {recall:.2f}")
    print(f"  F1-score: {f1:.2f}")


Evaluating: 100%|████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  5.92it/s]

Metrics for class Common wheat:
  Precision: 1.00
  Recall: 0.91
  F1-score: 0.95
Metrics for class Convolvulus arvensis:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
Metrics for class Cotton:
  Precision: 0.38
  Recall: 0.75
  F1-score: 0.50
Metrics for class Euphorbia peplus:
  Precision: 1.00
  Recall: 0.21
  F1-score: 0.35
Metrics for class Grass:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
Metrics for class Lolium multiflorum:
  Precision: 0.97
  Recall: 1.00
  F1-score: 0.98
Metrics for class Maize:
  Precision: 0.75
  Recall: 0.75
  F1-score: 0.75
Metrics for class Nutgrass:
  Precision: 0.96
  Recall: 0.96
  F1-score: 0.96
Metrics for class Purslane:
  Precision: 0.40
  Recall: 1.00
  F1-score: 0.57
Metrics for class Sesame:
  Precision: 0.91
  Recall: 1.00
  F1-score: 0.95
Metrics for class Sugar beet:
  Precision: 0.30
  Recall: 0.75
  F1-score: 0.43
Metrics for class Tomato:
  Precision: 1.00
  Recall: 0.43
  F1-score: 0.60


In [26]:
import torch
from torchvision import transforms
from PIL import Image

# Define the transformation for the input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load the trained model
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=11)
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

# Load the image
image_path = r"C:\Users\aliof\OneDrive\Desktop\tomatocut\0a9f2d5a-bc30-c5fd-4d6d-dc0b720e4b52.jpg"
image = Image.open(image_path)

# Apply transformations
input_image = transform(image).unsqueeze(0)  # Add a batch dimension

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_image = input_image.to(device)
model = model.to(device)

# Get predictions
with torch.no_grad():
    output = model(input_image)
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    predicted_class = torch.argmax(probabilities).item()

class_labels = ['Lolium_multiflorum', 'Maize', 'Nutgrass', 'Purslane', 'Convolvulus_arvensis', 
                'cotton', 'euphorbia', 'sesame', 'sugarbeet', 'tomato', 'wheat']

# Print prediction
print("Predicted Class:", class_labels[predicted_class])
print("Probability:", probabilities[predicted_class].item())


Loaded pretrained weights for efficientnet-b0
Predicted Class: euphorbia
Probability: 0.9999508857727051
